In [2]:
%%time
# coding: utf-8

# In[1]:


# !ls -hl|grep csv
import pandas as pd
import numpy as np
import os
from time import time, ctime

from sklearn.metrics import accuracy_score, classification_report, classification, confusion_matrix
from sklearn.model_selection import train_test_split, KFold, cross_val_score, StratifiedKFold, StratifiedShuffleSplit

dim=lambda *x: [i.shape for i in x]


# In[1]:


import os
import tensorflow as tf
from tensorflow.python.client import device_lib

CPU times: user 0 ns, sys: 0 ns, total: 0 ns
Wall time: 23.1 µs


In [5]:
import tensorflow as tf
from keras import backend as K

num_cores = 4

GPU=True
CPU=False

if GPU:
    num_GPU = 1
    num_CPU = 1
if CPU:
    num_CPU = 1
    num_GPU = 0

config = tf.ConfigProto(intra_op_parallelism_threads=num_cores,\
        inter_op_parallelism_threads=num_cores, allow_soft_placement=True,\
        device_count = {'CPU' : num_CPU, 'GPU' : num_GPU})
session = tf.Session(config=config)
K.set_session(session)

print(device_lib.list_local_devices())

[name: "/cpu:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 10881385210393610675
, name: "/gpu:0"
device_type: "GPU"
memory_limit: 11322956186
locality {
  bus_id: 2
}
incarnation: 9180730991985362561
physical_device_desc: "device: 0, name: Tesla K40m, pci bus id: 0000:82:00.0"
]


In [ ]:
%%time


from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator

import os


df = pd.read_csv('./ready.csv')
print (df.shape)
X=df.values

# In[6]:
y=pd.read_csv('./label_ready.csv')['label'].values


from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3,
                                                   random_state=7)


from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
# fit on training set
X_train = sc.fit_transform(X_train)
# only transform on test set
X_test = sc.transform(X_test)
print (dim(X_train,X_test, y_train, y_test))



In [9]:

classifier = Sequential()
# first hidden layer
classifier.add(Dense(units = 1024, 
                     input_dim=4491,
                     kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.8))
# second hidden layer
classifier.add(Dense(units = 512,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.7))
# second hidden layer
classifier.add(Dense(units = 256,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.6))
# thrid hidden layer
classifier.add(Dense(units = 128,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.5))
# thrid hidden layer
classifier.add(Dense(units = 64,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.4))
# thrid hidden layer
classifier.add(Dense(units = 32,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.3))
# thrid hidden layer
classifier.add(Dense(units = 32,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.2))
# thrid hidden layer
classifier.add(Dense(units = 64,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 128,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 256,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# thrid hidden layer
classifier.add(Dense(units = 1024,  kernel_initializer='uniform', activation='relu'))
classifier.add(Dropout(rate= 0.1))
# # ouput layer
classifier.add(Dense(units = 1,  kernel_initializer='uniform', activation='sigmoid'))
# compiling the ANN
classifier.compile(optimizer='adam',loss='binary_crossentropy', metrics= ['accuracy'])
print (classifier.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_20 (Dense)             (None, 1024)              4599808   
_________________________________________________________________
dropout_18 (Dropout)         (None, 1024)              0         
_________________________________________________________________
dense_21 (Dense)             (None, 512)               524800    
_________________________________________________________________
dropout_19 (Dropout)         (None, 512)               0         
_________________________________________________________________
dense_22 (Dense)             (None, 256)               131328    
_________________________________________________________________
dropout_20 (Dropout)         (None, 256)               0         
_________________________________________________________________
dense_23 (Dense)             (None, 128)               32896     
__________

In [10]:


from keras.callbacks import Callback, EarlyStopping
class TestCallback(Callback):
    def __init__(self, test_data):
        self.test_data = test_data
        

    def on_epoch_end(self, epoch, logs={}):
        x, y = self.test_data
        loss, acc = self.model.evaluate(x, y, verbose=1)
        print('\nTesting loss: {}, acc: {}\n'.format(loss, acc))


# In[15]:


stopping = EarlyStopping(monitor='acc', min_delta=0,
                              patience=6, verbose=1, mode='auto')


# In[16]:


# fitting ANN with training set
classifier.fit(X_train, y_train, 
               batch_size=64, epochs=64,
               validation_data=(X_test, y_test),
          callbacks=[TestCallback((X_test, y_test))])
#           callbacks=[stopping])
    


# In[53]:


y_pred = classifier.predict(X_test)
y_pred = (y_pred> 0.5)

print (accuracy_score(y_test, y_pred))
cm = confusion_matrix(y_test, y_pred)
print (cm)




Train on 34482 samples, validate on 14778 samples
Epoch 1/64
14656/14778 [============================>.] - ETA: 0s
Testing loss: 0.4770643329334801, acc: 0.8197997023246498

34482/34482 [==============================] - 9s - loss: 0.4776 - acc: 0.8193 - val_loss: 0.4771 - val_acc: 0.8198
Epoch 2/64
14144/14778 [===========================>..] - ETA: 0s
Testing loss: 0.4622755135843244, acc: 0.8197997023246498

34482/34482 [==============================] - 9s - loss: 0.4686 - acc: 0.8200 - val_loss: 0.4623 - val_acc: 0.8198
Epoch 3/64
14778/14778 [==============================] - 1s     

Testing loss: 0.4596625715554656, acc: 0.8197997023246498

34482/34482 [==============================] - 9s - loss: 0.4531 - acc: 0.8200 - val_loss: 0.4597 - val_acc: 0.8198
Epoch 4/64
14720/14778 [============================>.] - ETA: 0s
Testing loss: 0.4664444545719262, acc: 0.8197997023246498

34482/34482 [==============================] - 9s - loss: 0.4388 - acc: 0.8200 - val_loss: 0.4664 - v

34482/34482 [==============================] - 9s - loss: 0.1823 - acc: 0.9312 - val_loss: 0.6881 - val_acc: 0.7571
Epoch 35/64
14720/14778 [============================>.] - ETA: 0s
Testing loss: 0.7951851527977093, acc: 0.7599810529326309

34482/34482 [==============================] - 9s - loss: 0.1826 - acc: 0.9309 - val_loss: 0.7952 - val_acc: 0.7600
Epoch 36/64
14688/14778 [============================>.] - ETA: 0s
Testing loss: 0.6621343076091962, acc: 0.7502368386163814

34482/34482 [==============================] - 9s - loss: 0.1763 - acc: 0.9336 - val_loss: 0.6621 - val_acc: 0.7502
Epoch 37/64
14144/14778 [===========================>..] - ETA: 0s
Testing loss: 0.8221343897816485, acc: 0.7679658953142289

34482/34482 [==============================] - 9s - loss: 0.1762 - acc: 0.9338 - val_loss: 0.8221 - val_acc: 0.7680
Epoch 38/64
14208/14778 [===========================>..] - ETA: 0s
Testing loss: 0.7342788622333807, acc: 0.7385302477380487

34482/34482 [===================